## Imports

In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# from sklearn.linear_model import LogisticRegression #dont need
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier, plot_tree

## Load data

In [7]:
unprocessed_data = pd.read_csv("../data/raw/train.csv")

In [8]:
unprocessed_data.columns

Index(['CustomerID', 'Age', 'Gender', 'Tenure', 'Usage Frequency',
       'Support Calls', 'Payment Delay', 'Subscription Type',
       'Contract Length', 'Total Spend', 'Last Interaction', 'Customer Status',
       'Last Due Date', 'Last Payment Date', 'Churn'],
      dtype='object')

In [17]:
np.random.seed(311)
processed_data = unprocessed_data.dropna()
X = processed_data[["Age", "Gender", "Tenure", "Usage Frequency", "Support Calls", "Customer Status"]]
y = processed_data["Churn"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [15]:
X.head()

,Age,Gender,Tenure,Usage Frequency,Support Calls,Customer Status
0,47,Male,52.0,9,4,active
1,45,Female,13.0,16,1,active
2,33,Female,NaN,29,6,inactive
3,40,Male,13.0,25,3,active
4,38,Male,18.0,16,2,active


In [18]:
#try ph with depth 1, ph with depth 4, ph with depth None
#try N as the features with the same depth value
#try cycle through the rest of the fa
features = ["Age", "Usage Frequency", "Support Calls"]
depth_limit = 6
model = DecisionTreeClassifier(criterion='entropy', max_depth=depth_limit)
model.fit(X_train[features], y_train)

#Predict on train data
y_pred_train = model.predict(X_train[features])

#Predict on test data
y_pred_test = model.predict(X_test[features])

train_accuracy = metrics.accuracy_score(y_train, y_pred_train)
train_f1_score = metrics.f1_score(y_train, y_pred_train)
testing_accuracy = metrics.accuracy_score(y_test, y_pred_test)
testing_f1_score = metrics.f1_score(y_test, y_pred_test)

print("---------Training performance---------")
print(f"{train_accuracy=}")
print(f"{train_f1_score=}")
print("---------Testing performance---------")
print(f"{testing_accuracy=}")
print(f"{testing_f1_score=}")

plt.figure(figsize=(3,3))
plot_tree(model, feature_names=features, class_names=["coffee", "rice"], filled=True)
plt.title(f"Descision Tree ({features=}, {depth_limit=})")

ValueError: could not convert string to float: 'none'